## import our needs 


In [ ]:
import os 
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from sklearn.pipeline import Pipeline , FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics  import accuracy_score , make_scorer



## load the data 
you can load the data from here https://www.drivendata.org/accounts/login/?next=/competitions/57/nepal-earthquake/

In [5]:
path = # 'your data path'  
train_data = pd.read_csv(os.path.join(path , 'train_values.csv'))
train_labels = pd.read_csv(os.path.join(path , 'train_labels.csv'))

In [14]:
train_data.head()

array(['t', 's', 'j', 'o'], dtype=object)

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             260601 non-null  int64 
 1   geo_level_1_id                          260601 non-null  int64 
 2   geo_level_2_id                          260601 non-null  int64 
 3   geo_level_3_id                          260601 non-null  int64 
 4   count_floors_pre_eq                     260601 non-null  int64 
 5   age                                     260601 non-null  int64 
 6   area_percentage                         260601 non-null  int64 
 7   height_percentage                       260601 non-null  int64 
 8   land_surface_condition                  260601 non-null  object
 9   foundation_type                         260601 non-null  object
 10  roof_type                               260601 non-null 

## preprocess data 

In [24]:
# convert labels to zero index 
labels = train_labels['damage_grade'] - 1

In [35]:
# ourt custom class to select features from dataframe 
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names=attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values


In [17]:
# selet feature types 
features = train_data.columns 
numric_columns = features[1:8]
cats_columns = features[8:]

In [38]:
# our pipelines to process the data 
numric_pipeline = Pipeline([('selector' , DataFrameSelector(numric_columns)) 
                            , ('scaler' , StandardScaler())])
cat_pipeline = Pipeline([('selector',  DataFrameSelector(cats_columns)) , 
                         ('encoding' , OneHotEncoder())] )
data_pipeline = FeatureUnion(transformer_list = [
    ('numirc_pipeline' , numric_pipeline) , 
    ('cat_pipeline' , cat_pipeline)
])

In [46]:
preprocessed_data = data_pipeline.fit_transform(train_data)

In [48]:
# modeling your data 
scorer = make_scorer(accuracy_score)
rfc = RandomForestClassifier() 
cross_val = cross_val_score(rfc ,preprocessed_data , labels, scoring = scorer ) 
